In [3]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

data_path = "..//data//diabetes.csv"

df = pd.read_csv(data_path)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler 

X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = X.columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler())
]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ]
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
    
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
from sklearn.metrics import classification_report 

y_pred = pipeline.predict(X_test)

# model evaluation 
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.78      0.78        99
           1       0.61      0.62      0.61        55

    accuracy                           0.72       154
   macro avg       0.70      0.70      0.70       154
weighted avg       0.72      0.72      0.72       154



In [ ]:
# save the model 
import pickle 

with open("..//models//pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)

In [4]:
import requests 

url = "http://localhost:8000/predict"

data = {
    "data" : [[1, 85, 66, 29, 0, 26.6, 0.351, 31]]
}

response = requests.post(url, json=data)
print(response.json())

{'detail': 'np.int64(0)'}
